# Recommended system

## Q1 Generate an audience of shoppers that would be ideal for each of the potential offers.
1. How did you identify these users?
2. How would you handle users that are relatively new to the Fetch platform and don't have a lengthy transaction history?
3. How would you explain your approach to a partner brand?

## Q2  2. If every single customer in the historical transaction period was to be shown only three offers from the list of potential offers, design/describe/build an approach that would allow you to choose the best three and rank them per user.
1. How did you choose the best three offers per user?
2. How did you choose the ranking?
3. How would you test your approach to know if it was effective?

## answer1

How did you identify these users? <br>

First, I made up the three categories (BABY,HOUSEHOLD,PERSONAL_CARE) in transcation data. 
Second, I built up a user-item matrix based on the user expense in each category and brand. The matrix value is the total moeny a user spending in each category of each brand.
Third, I chose the top buyer of each category and brand.
Fourth, I used cosine similarity to calculate the cos value between the top buyer and each user. 
Fifth, I ranked the users by using the cosine value, then pick up top 10 as the users for the special offers of each brand and each category. <br>

How would you handle users that are relatively new to the Fetch platform and don't have a lengthy transaction history? <br>

This question is related to cold start issue.
If the user doesn't have a long-term transaction history, I can just promote that user the top 3 popular products in each brand and each category.
First, I calculated the number of sold items of each product for each brand.
Second, I ranked the product for each brand based on the number of sold items.
Third, I listed the top 3 popular products in each brand. <br>

How would you explain your approach to a partner brand? <br>

My approach to promote the special offer for each brand and each category is based on user's purchase habits. That is promoting
special offers of one brand in one category to a user group, which is similiar to the top buyers. It avoids the external competition between each brand. <br>


## answer2

How did you choose the best three offers per user? <br>
How did you choose the ranking? <br>

First, I built up a item-user matrix. And calculated the brand and the category that each user spending most money in. 
Second, I randomly selected three offers from the special offers from the brand and category that the user spending most money in.  <br>

The ranking of special offers depends on the popularity of the product. If user A bought more product 1 than product 2,
then the special offer of product 1 is higher ranked than product 2 for the user A.  <br>

How would you test your approach to know if it was effective? <br>

Since it is a time-series data, to test my model, I would like to use the data of April and May as the training data 
and the data of June as the testing data. 
If the bought items of product 1 in June is more than the mean of bought items of product 1 of in April and May for user A,
I consider it as a good promotion and count for 1.
If not, then count for 0.
Then, I calculated the count of good promotions divided by the total of sold products as the effectiveness for my model. <br>

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import warnings
warnings.filterwarnings('ignore')

## answer1

In [2]:
#load the data and check the data info
transaction_df=pd.read_csv(r'data\raw\transaction_data_simple.csv')
special_df=pd.read_csv(r'data\raw\special_offers_simple.csv')

In [3]:
transaction_df.columns

Index(['Unnamed: 0', 'purchase_date', 'transaction_id', 'receipt_total',
       'user_id', 'product_id', 'brand', 'category_1', 'category_2',
       'offer_redemption', 'item_quantity', 'item_unit_price'],
      dtype='object')

In [4]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612626 entries, 0 to 612625
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        612626 non-null  int64  
 1   purchase_date     612626 non-null  object 
 2   transaction_id    612626 non-null  uint64 
 3   receipt_total     612626 non-null  float64
 4   user_id           612626 non-null  uint64 
 5   product_id        612626 non-null  uint64 
 6   brand             612626 non-null  uint64 
 7   category_1        612626 non-null  object 
 8   category_2        612626 non-null  object 
 9   offer_redemption  612626 non-null  int64  
 10  item_quantity     612626 non-null  float64
 11  item_unit_price   612626 non-null  float64
dtypes: float64(3), int64(2), object(3), uint64(4)
memory usage: 56.1+ MB


In [5]:
special_df.columns

Index(['Unnamed: 0', 'offer_id', 'product_id', 'category', 'cents_required',
       'allows_multitransaction', 'start_date', 'end_date', 'duration',
       'brands'],
      dtype='object')

In [6]:
special_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4865 entries, 0 to 4864
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               4865 non-null   int64  
 1   offer_id                 4865 non-null   object 
 2   product_id               4865 non-null   uint64 
 3   category                 4865 non-null   object 
 4   cents_required           776 non-null    float64
 5   allows_multitransaction  4865 non-null   bool   
 6   start_date               4865 non-null   object 
 7   end_date                 4865 non-null   object 
 8   duration                 4865 non-null   int64  
 9   brands                   4865 non-null   uint64 
dtypes: bool(1), float64(1), int64(2), object(4), uint64(2)
memory usage: 346.9+ KB


In [7]:
#explore brands and categories
t_cate=transaction_df.groupby(["category_1","category_2"])["category_2"].count()
s_cate=special_df.groupby("category")["category"].count()
t_br=transaction_df.groupby("brand")["brand"].count()
s_br=special_df.groupby("brands")["brands"].count()
print("transaction_data_cate: ",t_cate)
print("\nspecial_offer_cate: ",s_cate)
print("\ntransaction_data_brand: ",t_br)
print("\nspecial_offer_brand: ",s_br)

transaction_data_cate:  category_1          category_2              
ADULT INCONTINENCE  ADULT BED UNDERPAD              6
                    ADULT BLADDER SUPPORT         142
                    ADULT BRIEF                  2015
                    ADULT FURNITURE UNDERPAD      151
                    ADULT GUARD                   733
                    ADULT LINER                 11267
                    ADULT PAD                   22985
                    ADULT UNDERWEAR              8484
BATH TISSUE         BATH TISSUE PREMIUM         23626
                    BATH TISSUE VALUE           90892
DIAPERS             AB                          56053
                    POME                        88926
FACIAL TISSUE       FACIAL TISSUE PREMIUM       59248
FEMININE CARE       ADULT PAD                     187
                    DISPOSABLE UNDERWEAR            7
                    FEMININE WIPE                   4
                    PADS                        24821
             

In [8]:
# some products in the special offer are new to the transaction data
num_product_tran=np.unique(transaction_df['product_id'])
len(num_product_tran)

1825

In [9]:
num_product_spec=np.unique(special_df['product_id'])
len(num_product_spec)

2836

In [10]:
#data clean
special_data=special_df
special_data.loc[special_data['category']=='Household','category']='HOUSEHOLD'
special_data['start_date'] = pd.to_datetime(special_data['start_date'])
special_data['end_date'] = pd.to_datetime(special_data['end_date'])

In [11]:
print(special_data['start_date'].min(),special_data['start_date'].max())

2022-07-01 00:00:00 2022-09-01 00:00:00


In [12]:
transaction_data=transaction_df
transaction_data['purchase_date'] = pd.to_datetime(transaction_data['purchase_date'])
transaction_data['cate_match']=0
transaction_data.loc[transaction_data['category_1']=='ADULT INCONTINENCE',['cate_match']]='PERSONAL_CARE'
transaction_data.loc[transaction_data['category_1']=='BATH TISSUE',['cate_match']]='HOUSEHOLD'
transaction_data.loc[transaction_data['category_1']=='DIAPERS',['cate_match']]='BABY'
transaction_data.loc[transaction_data['category_1']=='FACIAL TISSUE',['cate_match']]='PERSONAL_CARE'
transaction_data.loc[transaction_data['category_1']=='FEMININE CARE',['cate_match']]='PERSONAL_CARE'
transaction_data.loc[transaction_data['category_1']=='PAPER TOWEL',['cate_match']]='HOUSEHOLD'
transaction_data.loc[transaction_data['category_1']=='TOTAL WIPES',['cate_match']]='HOUSEHOLD'
transaction_data.loc[transaction_data['category_1']=='SWIM PANTS',['cate_match']]='BABY'
transaction_data.loc[transaction_data['category_1']=='TRAINING PANTS',['cate_match']]='BABY'
transaction_data.loc[transaction_data['category_1']=='YOUTH PANTS',['cate_match']]='BABY'

In [13]:
print(transaction_data['purchase_date'].min(),transaction_data['purchase_date'].max())

2022-04-01 00:00:00 2022-06-30 00:00:00


In [14]:
# select the user of spending most money on one product
transaction_data['item_total']=transaction_data['item_quantity']*transaction_data['item_unit_price']
user_data=transaction_data.groupby(['transaction_id','product_id','user_id','brand','cate_match'])['item_total'].sum().reset_index()

In [15]:
p_table = pd.pivot_table(user_data, values='item_total', index=['user_id'], columns=['cate_match', 'brand'], aggfunc="sum", fill_value=0)

In [16]:
p_table

cate_match                           BABY                       \
brand                1762597501371776412  13388685157089796813   
user_id                                                          
82845826737446                       0.00                  0.0   
111363591852008                      0.00                  0.0   
111764929589779                      0.00                  0.0   
221659438151809                      0.00                  0.0   
226216623368547                      0.00                  0.0   
...                                   ...                  ...   
18446561187805844330                 2.39                  0.0   
18446661983785285690                 0.00                  0.0   
18446664432442808716                 0.00                  0.0   
18446694276717023506                 0.00                  0.0   
18446719638191844279                 0.00                  0.0   

cate_match                                           HOUSEHOLD  \
brand                16209527524158134138 1762597501371776412    
user_id                                                          
82845826737446                        0.0                 0.00   
111363591852008                       0.0                 0.00   
111764929589779                       0.0                 0.00   
221659438151809                       0.0                 0.00   
226216623368547                       0.0                 0.00   
...                                   ...                  ...   
18446561187805844330                  0.0                 0.11   
18446661983785285690                  0.0                 0.00   
18446664432442808716                  0.0                 1.92   
18446694276717023506                  0.0                 0.00   
18446719638191844279                  0.0                 0.00   

cate_match                                                      \
brand                3427381628801314608  3447117117351085285    
user_id                                                          
82845826737446                        0.0                  0.0   
111363591852008                       0.0                  0.0   
111764929589779                       0.0                  0.0   
221659438151809                       0.0                  0.0   
226216623368547                       0.0                  0.0   
...                                   ...                  ...   
18446561187805844330                  0.0                  0.0   
18446661983785285690                  0.0                  0.0   
18446664432442808716                  0.0                  0.0   
18446694276717023506                  0.0                  0.0   
18446719638191844279                  0.0                  0.0   

cate_match                                                      \
brand                3828245197284552407  11021339065701790727   
user_id                                                          
82845826737446                       0.00                 4.95   
111363591852008                      0.00                 0.00   
111764929589779                     13.99                 0.00   
221659438151809                      0.00                 3.63   
226216623368547                      0.00                 0.00   
...                                   ...                  ...   
18446561187805844330                 0.00                 0.00   
18446661983785285690                 8.99                 0.00   
18446664432442808716                 0.00                 0.00   
18446694276717023506                 0.00                39.98   
18446719638191844279                 0.00                10.95   

cate_match                                                      \
brand                11548329967324919662 13388685157089796813   
user_id                                                          
82845826737446                       0.00                  0.0   
111363591852008                     

In [17]:
cate_list=p_table.columns.to_list()
cate_list

[('BABY', 1762597501371776412),
 ('BABY', 13388685157089796813),
 ('BABY', 16209527524158134138),
 ('HOUSEHOLD', 1762597501371776412),
 ('HOUSEHOLD', 3427381628801314608),
 ('HOUSEHOLD', 3447117117351085285),
 ('HOUSEHOLD', 3828245197284552407),
 ('HOUSEHOLD', 11021339065701790727),
 ('HOUSEHOLD', 11548329967324919662),
 ('HOUSEHOLD', 13388685157089796813),
 ('PERSONAL_CARE', 1279604269096745345),
 ('PERSONAL_CARE', 3447117117351085285),
 ('PERSONAL_CARE', 6966397462150644289),
 ('PERSONAL_CARE', 13774051611309538603),
 ('PERSONAL_CARE', 14959283507241671213)]

In [18]:
cate_fav=[]
for i in range(len(cate_list)):
    idx=p_table.index[p_table[cate_list[i]]==p_table[cate_list[i]].max()].tolist()
    cate_fav.append([cate_list[i],idx])

In [19]:
cate_fav

[[('BABY', 1762597501371776412), [12739588705792971190]],
 [('BABY', 13388685157089796813), [17975070263254539184]],
 [('BABY', 16209527524158134138), [18093103750259743376]],
 [('HOUSEHOLD', 1762597501371776412), [345733402809375563]],
 [('HOUSEHOLD', 3427381628801314608), [2675490404890151794]],
 [('HOUSEHOLD', 3447117117351085285),
  [8449637223256584705, 10506094684453501535]],
 [('HOUSEHOLD', 3828245197284552407), [14842792811782633616]],
 [('HOUSEHOLD', 11021339065701790727), [13597453118102940595]],
 [('HOUSEHOLD', 11548329967324919662), [2797391074475694508]],
 [('HOUSEHOLD', 13388685157089796813), [13303717485415785443]],
 [('PERSONAL_CARE', 1279604269096745345), [10334096057175679930]],
 [('PERSONAL_CARE', 3447117117351085285), [3194949510063462734]],
 [('PERSONAL_CARE', 6966397462150644289),
  [4375865735292781979, 13811899413676258444]],
 [('PERSONAL_CARE', 13774051611309538603), [14261772673231086290]],
 [('PERSONAL_CARE', 14959283507241671213), [961428233505720394]]]

In [139]:
from sklearn import metrics

user_cos=np.zeros((len(p_table),len(p_table.columns)))

for i in range(len(cate_list)):
    X=p_table.loc[p_table.index==cate_fav[i][1][0],:].to_numpy()

    for j in range(len(p_table)):
        Y=p_table.iloc[j].values
        Y=Y.reshape((1,-1))
        cos=sklearn.metrics.pairwise.cosine_similarity(X, Y, dense_output=True)
        user_cos[j,i]=cos[0,0]

In [147]:
user_df = pd.DataFrame(user_cos, columns=p_table.columns, index=p_table.index)
user_df

cate_match                           BABY                       \
brand                1762597501371776412  13388685157089796813   
user_id                                                          
82845826737446                   0.000000                  0.0   
111363591852008                  0.000000                  0.0   
111764929589779                  0.000000                  0.0   
221659438151809                  0.000000                  0.0   
226216623368547                  0.000000                  0.0   
...                                   ...                  ...   
18446561187805844330             0.998943                  0.0   
18446661983785285690             0.000000                  0.0   
18446664432442808716             0.000000                  0.0   
18446694276717023506             0.000000                  0.0   
18446719638191844279             0.000000                  0.0   

cate_match                                           HOUSEHOLD  \
brand                16209527524158134138 1762597501371776412    
user_id                                                          
82845826737446                        0.0             0.000000   
111363591852008                       0.0             0.000000   
111764929589779                       0.0             0.000000   
221659438151809                       0.0             0.000000   
226216623368547                       0.0             0.000000   
...                                   ...                  ...   
18446561187805844330                  0.0             0.045976   
18446661983785285690                  0.0             0.000000   
18446664432442808716                  0.0             1.000000   
18446694276717023506                  0.0             0.000000   
18446719638191844279                  0.0             0.000000   

cate_match                                                      \
brand                3427381628801314608  3447117117351085285    
user_id                                                          
82845826737446                        0.0                  0.0   
111363591852008                       0.0                  0.0   
111764929589779                       0.0                  0.0   
221659438151809                       0.0                  0.0   
226216623368547                       0.0                  0.0   
...                                   ...                  ...   
18446561187805844330                  0.0                  0.0   
18446661983785285690                  0.0                  0.0   
18446664432442808716                  0.0                  0.0   
18446694276717023506                  0.0                  0.0   
18446719638191844279                  0.0                  0.0   

cate_match                                                      \
brand                3828245197284552407  11021339065701790727   
user_id                                                          
82845826737446                        0.0                  1.0   
111363591852008                       0.0                  0.0   
111764929589779                       1.0                  0.0   
221659438151809                       0.0                  1.0   
226216623368547                       0.0                  0.0   
...                                   ...                  ...   
18446561187805844330                  0.0                  0.0   
18446661983785285690                  1.0                  0.0   
18446664432442808716                  0.0                  0.0   
18446694276717023506                  0.0                  1.0   
18446719638191844279                  0.0                  1.0   

cate_match                                                      \
brand                11548329967324919662 13388685157089796813   
user_id                                                          
82845826737446                        0.0                  0.0   
111363591852008                     

In [158]:
offer_index=pd.DataFrame(special_data.groupby(['category','brands']).count()).index

In [167]:
user_dict={}
for i in range(len(offer_index)):
    user_dict[offer_index[i]]=user_df.nlargest(10, offer_index[i]).index.tolist()
user_dict

{('BABY', 1762597501371776412): [651885771758456,
  785822639486653,
  854928476555369,
  1234273649149620,
  1306343002939569,
  1774367149489052,
  1988775371333357,
  2019904797443736,
  3374190764118012,
  3860186291055296],
 ('BABY', 13388685157089796813): [2321876113835592,
  2794119879663443,
  4408748771769203,
  5475894427707559,
  8401968744414148,
  8734225932073711,
  9299066663119839,
  9959502090687463,
  10025210881661951,
  10704962499024264],
 ('BABY', 16209527524158134138): [520094740242410,
  7809910569982378,
  8197660702650581,
  11062716806776390,
  11944205077567589,
  14097110608080088,
  18106015894335107,
  18434188256351663,
  19609764969290286,
  23065918545728098],
 ('HOUSEHOLD', 3447117117351085285): [7883999523474411,
  12003569012423203,
  19443490175990848,
  49671592897228691,
  57841929582299038,
  63967196879246976,
  98023055401739238,
  158603191930413661,
  172894266428702427,
  177457985935188581],
 ('HOUSEHOLD', 11021339065701790727): [828458267

In [208]:
special_data['user']=''
special_data['combo']=''
key_list=list(user_dict.keys())
value_list=list(user_dict.values())

In [213]:
for i in range(len(special_data)):
    special_data.combo.iloc[i]=tuple([special_data.category.iloc[i],special_data.brands.iloc[i]])
    for j in range(len(user_dict)):
        if special_data.combo.iloc[i]==key_list[j]:
            special_data.user.iloc[i]=value_list[j]

In [214]:
special_data

,Unnamed: 0,offer_id,product_id,category,cents_required,allows_multitransaction,start_date,end_date,duration,brands,user,combo
0,17656,01G5PWX9YS3ZTZ87EEVY70KVX6,4899226240041671261,BABY,NaN,False,2022-07-01,2022-08-31 23:59:59.999,61,1762597501371776412,"[651885771758456, 785822639486653, 85492847655...","(BABY, 1762597501371776412)"
1,10620,01G7WH06537QAZM4T1DXAYKV0M,1941647917893097381,PERSONAL_CARE,NaN,False,2022-08-01,2022-08-31 23:59:59.999,31,13774051611309538603,"[14261772673231086290, 8660754902732730032, 75...","(PERSONAL_CARE, 13774051611309538603)"
2,27450,01GAHCYYR8VR7KC4R03SZS2Y7E,6865308734032154080,BABY,NaN,False,2022-09-01,2022-09-30 23:59:59.999,30,16209527524158134138,"[520094740242410, 7809910569982378, 8197660702...","(BABY, 16209527524158134138)"
3,30688,01GAP87DXVP527JNEGTSJ0FBX3,10418096192768067372,BABY,6000.0,True,2022-09-01,2022-09-30 23:59:59.999,30,1762597501371776412,"[651885771758456, 785822639486653, 85492847655...","(BABY, 1762597501371776412)"
4,12765,01G5EYJZCDTXN8C5J4MAEK3X5A,8414116070997053836,HOUSEHOLD,NaN,True,2022-07-01,2022-08-31 23:59:59.999,62,11021339065701790727,"[82845826737446, 221659438151809, 756797749278...","(HOUSEHOLD, 11021339065701790727)"
...,...,...,...,...,...,...,...,...,...,...,...,...
4860,25935,01GA6PXDXQD07T52G65KHVH8MZ,13458863303946004617,PERSONAL_CARE,NaN,False,2022-09-01,2022-09-30 23:59:59.999,30,13774051611309538603,"[14261772673231086290, 8660754902732730032, 75...","(PERSONAL_CARE, 13774051611309538603)"
4861,44466,01G52SJZC76H1AA8MAW4CNAVMV,12402683988788894059,PERSONAL_CARE,NaN,False,2022-07-01,2022-07-31 23:59:59.999,31,13774051611309538603,"[14261772673231086290, 8660754902732730032, 75...","(PERSONAL_CARE, 13774051611309538603)"
4862,11956,01G5EPJCAZKKJJYRNMCHE40894,2255018128531149269,HOUSEHOLD,NaN,False,2022-07-01,2022-07-31 23:59:59.999,31,11548329967324919662,"[111363591852008, 226216623368547, 10928570327...","(HOUSEHOLD, 11548329967324919662)"
4863,40407,01GAS4AY2AAX007T3MBWFACH0M,13760208613540948776,HOUSEHOLD,NaN,False,2022-09-01,2022-09-30 23:59:59.999,30,3447117117351085285,"[7883999523474411, 12003569012423203, 19443490...","(HOUSEHOLD, 3447117117351085285)"


In [20]:
nu_products=transaction_data.loc[:,['brand','product_id','item_quantity']]
nu_product_br=nu_products.groupby(['brand','product_id'])['item_quantity'].sum().reset_index()
nu_product_br['rk']=nu_product_br.groupby(['brand'])['item_quantity'].rank(method='first',ascending=False)

nu_product_brf=nu_product_br.loc[nu_product_br['rk']<=3.0,:]
print('pop products in each brand: \n', nu_product_brf.sort_values(by=['brand','rk']))

pop products in each brand: 
                      brand            product_id  item_quantity   rk
64     1279604269096745345  10552228047546454307         2214.0  1.0
32     1279604269096745345   5445359571079376581         2137.0  2.0
36     1279604269096745345   6672724103166384541         2125.0  3.0
399    1762597501371776412   9668595804925383893        40730.0  1.0
641    1762597501371776412  17452210309957977774        16024.0  2.0
347    1762597501371776412   7546349628703587271        15414.0  3.0
670    3427381628801314608  16956431849410263386            6.0  1.0
669    3427381628801314608    476007654581878853            1.0  2.0
699    3447117117351085285   1611083189200273707         3719.0  1.0
727    3447117117351085285   4168593371591170658         3075.0  2.0
839    3447117117351085285  12326650824606238900         2812.0  3.0
969    3828245197284552407   9625919461608592222         2307.0  1.0
934    3828245197284552407    792131708893121653         2183.0  2.0
967 

## answer 2

In [21]:
p_table_re = pd.pivot_table(user_data, values='item_total', index=['cate_match', 'brand'], columns=['user_id'], aggfunc="sum", fill_value=0)
p_table_re

user_id                             82845826737446        \
cate_match    brand                                        
BABY          1762597501371776412                   0.00   
              13388685157089796813                  0.00   
              16209527524158134138                  0.00   
HOUSEHOLD     1762597501371776412                   0.00   
              3427381628801314608                   0.00   
              3447117117351085285                   0.00   
              3828245197284552407                   0.00   
              11021339065701790727                  4.95   
              11548329967324919662                  0.00   
              13388685157089796813                  0.00   
PERSONAL_CARE 1279604269096745345                   0.00   
              3447117117351085285                   0.00   
              6966397462150644289                   0.00   
              13774051611309538603                  0.00   
              14959283507241671213                  0.00   

user_id                             111363591852008       \
cate_match    brand                                        
BABY          1762597501371776412                   0.00   
              13388685157089796813                  0.00   
              16209527524158134138                  0.00   
HOUSEHOLD     1762597501371776412                   0.00   
              3427381628801314608                   0.00   
              3447117117351085285                   0.00   
              3828245197284552407                   0.00   
              11021339065701790727                  0.00   
              11548329967324919662                  4.25   
              13388685157089796813                  0.00   
PERSONAL_CARE 1279604269096745345                   0.00   
              3447117117351085285                   0.00   
              6966397462150644289                   0.00   
              13774051611309538603                  0.00   
              14959283507241671213                  0.00   

user_id                             111764929589779       \
cate_match    brand                                        
BABY          1762597501371776412                   0.00   
              13388685157089796813                  0.00   
              16209527524158134138                  0.00   
HOUSEHOLD     1762597501371776412                   0.00   
              3427381628801314608                   0.00   
              3447117117351085285                   0.00   
              3828245197284552407                  13.99   
              11021339065701790727                  0.00   
              11548329967324919662                  0.00   
              13388685157089796813                  0.00   
PERSONAL_CARE 1279604269096745345                   0.00   
              3447117117351085285                   0.00   
              6966397462150644289                   0.00   
              13774051611309538603                  0.00   
              14959283507241671213                  0.00   

user_id                             221659438151809       \
cate_match    brand                                        
BABY          1762597501371776412                   0.00   
              13388685157089796813                  0.00   
              16209527524158134138                  0.00   
HOUSEHOLD     1762597501371776412                   0.00   
              3427381628801314608                   0.00   
              3447117117351085285                   0.00   
              3828245197284552407                   0.00   
              11021339065701790727                  3.63   
              11548329967324919662                  0.00   
              13388685157089796813                  0.00   
PERSONAL_CARE 1279604269096745345                   0.00   
              3447117117351085285                   0.00   
              6966397462150644289                   0.00   
              13774051611309538603   

In [22]:
user_list=p_table_re.columns.to_list()

In [23]:
user_fav=[]
for i in range(len(user_list)):
    idx=p_table_re.index[p_table_re[user_list[i]]==p_table_re[user_list[i]].max()].tolist()
    user_fav.append([user_list[i],idx])

In [24]:
user_fav[0][1][0]

('HOUSEHOLD', 11021339065701790727)

In [27]:
offer_group=special_data.groupby(['category','brands','offer_id']).count().reset_index()
offer_group=offer_group.sort_values(['category','brands','product_id'],ascending=[True,True,False])
offer_group['rk']=offer_group.groupby(['category','brands'])['product_id'].rank('first',ascending=False)
offer_group

,category,brands,offer_id,Unnamed: 0,product_id,cents_required,allows_multitransaction,start_date,end_date,duration,rk
9,BABY,1762597501371776412,01GAP87DXVP527JNEGTSJ0FBX3,143,143,143,143,143,143,143,1.0
10,BABY,1762597501371776412,01GAP9YGGRQCX7JVZPE73T595T,136,136,0,136,136,136,136,2.0
6,BABY,1762597501371776412,01G7W3TCRX5EYH17HFT0YW2VNY,123,123,123,123,123,123,123,3.0
0,BABY,1762597501371776412,01G5PVXWZ4GZ5CX8XD6A24DMVP,120,120,0,120,120,120,120,4.0
2,BABY,1762597501371776412,01G5PWD1B0124QR8E3XW9Z4ZAK,108,108,0,108,108,108,108,5.0
...,...,...,...,...,...,...,...,...,...,...,...
201,PERSONAL_CARE,14959283507241671213,01G52V4C4C0ZVKM2YX7FEF5YRB,2,2,0,2,2,2,2,1.0
202,PERSONAL_CARE,14959283507241671213,01G52V7FTGXFNT4R4PK4TMCKH0,2,2,0,2,2,2,2,2.0
203,PERSONAL_CARE,14959283507241671213,01GA9CE677Y84CEF5RC4RE27RY,1,1,0,1,1,1,1,3.0
204,PERSONAL_CARE,14959283507241671213,01GA9EBRH564S0Z45KFJKXXV88,1,1,0,1,1,1,1,4.0


In [28]:
offer_group3=offer_group.loc[offer_group['rk']<=3,:]
offer_group3['combo']=''
for i in range(len(offer_group3)):
    offer_group3.combo.iloc[i]=tuple([offer_group3.category.iloc[i],offer_group3.brands.iloc[i]])
offer_group3

,category,brands,offer_id,Unnamed: 0,product_id,cents_required,allows_multitransaction,start_date,end_date,duration,rk,combo
9,BABY,1762597501371776412,01GAP87DXVP527JNEGTSJ0FBX3,143,143,143,143,143,143,143,1.0,"(BABY, 1762597501371776412)"
10,BABY,1762597501371776412,01GAP9YGGRQCX7JVZPE73T595T,136,136,0,136,136,136,136,2.0,"(BABY, 1762597501371776412)"
6,BABY,1762597501371776412,01G7W3TCRX5EYH17HFT0YW2VNY,123,123,123,123,123,123,123,3.0,"(BABY, 1762597501371776412)"
27,BABY,13388685157089796813,01G5S5CVD7HK2XK1Y9WYZ4KXV3,62,62,0,62,62,62,62,1.0,"(BABY, 13388685157089796813)"
62,BABY,13388685157089796813,01GARZ23W1DQAYVPJGZNX6WPM5,62,62,0,62,62,62,62,2.0,"(BABY, 13388685157089796813)"
65,BABY,13388685157089796813,01GARZXJWAANTS6YFBYJWHHYYJ,60,60,0,60,60,60,60,3.0,"(BABY, 13388685157089796813)"
69,BABY,16209527524158134138,01G54M5YD95F4QRDWNQTM88SAK,22,22,0,22,22,22,22,1.0,"(BABY, 16209527524158134138)"
75,BABY,16209527524158134138,01G7WCJGX8ZVWAJ5P7QSJS556Z,18,18,0,18,18,18,18,2.0,"(BABY, 16209527524158134138)"
86,BABY,16209527524158134138,01GAHCYYR8VR7KC4R03SZS2Y7E,18,18,0,18,18,18,18,3.0,"(BABY, 16209527524158134138)"
92,HOUSEHOLD,3447117117351085285,01G5Q7VWVGP7M2KT265MF7PXBM,67,67,67,67,67,67,67,1.0,"(HOUSEHOLD, 3447117117351085285)"


In [29]:
name_list=np.unique(offer_group3['combo'])
name_list

array([('BABY', 1762597501371776412), ('BABY', 13388685157089796813),
       ('BABY', 16209527524158134138), ('HOUSEHOLD', 3447117117351085285),
       ('HOUSEHOLD', 11021339065701790727),
       ('HOUSEHOLD', 11548329967324919662),
       ('PERSONAL_CARE', 1279604269096745345),
       ('PERSONAL_CARE', 6966397462150644289),
       ('PERSONAL_CARE', 13774051611309538603),
       ('PERSONAL_CARE', 14959283507241671213)], dtype=object)

In [31]:
offer_list=[]
for i in range(len(name_list)):
    temp=[]
    for j in range(len(offer_group3)):
        if name_list[i]==offer_group3.combo.iloc[j]:
            temp.append(offer_group3.offer_id.iloc[j])
    offer_list.append(temp)

In [32]:
offer_dict={}
for i in range(len(name_list)):
    offer_dict[name_list[i]]=offer_list[i]
offer_dict

{('BABY', 1762597501371776412): ['01GAP87DXVP527JNEGTSJ0FBX3',
  '01GAP9YGGRQCX7JVZPE73T595T',
  '01G7W3TCRX5EYH17HFT0YW2VNY'],
 ('BABY', 13388685157089796813): ['01G5S5CVD7HK2XK1Y9WYZ4KXV3',
  '01GARZ23W1DQAYVPJGZNX6WPM5',
  '01GARZXJWAANTS6YFBYJWHHYYJ'],
 ('BABY', 16209527524158134138): ['01G54M5YD95F4QRDWNQTM88SAK',
  '01G7WCJGX8ZVWAJ5P7QSJS556Z',
  '01GAHCYYR8VR7KC4R03SZS2Y7E'],
 ('HOUSEHOLD', 3447117117351085285): ['01G5Q7VWVGP7M2KT265MF7PXBM',
  '01GAS5A9Q15JJXNYC0A3QSD328',
  '01GAS4AY2AAX007T3MBWFACH0M'],
 ('HOUSEHOLD', 11021339065701790727): ['01G5EYJZCDTXN8C5J4MAEK3X5A',
  '01G5EY6RFAZJJD09V2D708G8Q5',
  '01G7D4PNYVFFP2H1X2ST5HGRV4'],
 ('HOUSEHOLD', 11548329967324919662): ['01G5EN6BETJYC0WNG4HWH9QG83',
  '01G5EMZP9YTZXG77J15RT1B6SB',
  '01G7CPS77PGBYRE3XCW9HEEZPZ'],
 ('PERSONAL_CARE', 1279604269096745345): ['01GA9E6QFK6264EF5T1TVPH2VD',
  '01G8XJTZ5QG0MN16V4S2FNRD0R',
  '01G52TD1K8595ZHFWSTZKGKVMD'],
 ('PERSONAL_CARE', 6966397462150644289): ['01G52Q8XSRZTAYFYFPVE7XBP0Z',
  '0

In [33]:
offer_list_key=list(offer_dict.keys())
offer_list_value=list(offer_dict.values())

In [34]:
for i in range(len(user_fav)):
    for j in range(len(offer_dict)):
        if user_fav[i][1][0]==offer_list_key[j]:
            user_fav[i].append(offer_list_value[j])

In [35]:
offer_df=pd.DataFrame(user_fav,columns=['user_id','category-brand','offer_id'])
offer_df

,user_id,category-brand,offer_id
0,82845826737446,"[(HOUSEHOLD, 11021339065701790727)]","[01G5EYJZCDTXN8C5J4MAEK3X5A, 01G5EY6RFAZJJD09V..."
1,111363591852008,"[(HOUSEHOLD, 11548329967324919662)]","[01G5EN6BETJYC0WNG4HWH9QG83, 01G5EMZP9YTZXG77J..."
2,111764929589779,"[(HOUSEHOLD, 3828245197284552407)]",None
3,221659438151809,"[(HOUSEHOLD, 11021339065701790727)]","[01G5EYJZCDTXN8C5J4MAEK3X5A, 01G5EY6RFAZJJD09V..."
4,226216623368547,"[(HOUSEHOLD, 11548329967324919662)]","[01G5EN6BETJYC0WNG4HWH9QG83, 01G5EMZP9YTZXG77J..."
...,...,...,...
438231,18446561187805844330,"[(BABY, 1762597501371776412)]","[01GAP87DXVP527JNEGTSJ0FBX3, 01GAP9YGGRQCX7JVZ..."
438232,18446661983785285690,"[(HOUSEHOLD, 3828245197284552407)]",None
438233,18446664432442808716,"[(HOUSEHOLD, 1762597501371776412)]",None
438234,18446694276717023506,"[(HOUSEHOLD, 11021339065701790727)]","[01G5EYJZCDTXN8C5J4MAEK3X5A, 01G5EY6RFAZJJD09V..."
